In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from bs4 import BeautifulSoup
import re
import pymorphy2
from pymorphy2.tagset import OpencorporaTag
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (25,10)

/Users/Vitalina/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv('train.csv', encoding='utf-8', sep='\t')
target = df_train['target']
df_train = df_train.drop('target', axis=1)

In [3]:
# морфологический анализатор и опредление англоязычных слов
ma = pymorphy2.MorphAnalyzer()
latin = OpencorporaTag('LATN')

In [4]:
# Морфологический кеш
parse_results = {}

def get_parse_result(word):
    # Приведем в нижний регистр
    word = word.lower()

    # Если слова нет в кеше - разбираем и записываем в кеш
    if not (word in parse_results):
        pv = ma.parse(word)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB'] or p.tag == latin:
                parse_results[word] = p.normal_form
                break

    # Если слово не было латиницей или нужной частью речи, вернем None
    if not (word in parse_results):
        parse_results[word] = None

    return parse_results[word]

In [5]:
# получаем слова
def getMeaningfullWords(text):
    meaning_words = []

    # заменяем тексты в скобках на пустоту
    clean_text = re.sub('([\(]).*?([\)])', '', text)

    # находим слова 3+ символов
    all_words = re.findall('[А-ЯЁа-яёA-Za-z]{3,}', clean_text)

    for word in all_words:
        parse_result = get_parse_result(word)
        if parse_result is not None:
            meaning_words.append(parse_result)

    return meaning_words

In [6]:
deleted_words = [
    'Обязанности:', 
    'Требования:', 
    'Условия:', 
    'Что нужно энергичным людям? Интересная и высокооплачиваемая работа в компании единомышленников!',
    'Для каждого, кто хочет работать и зарабатывать!Для каждого, кто готов к профессиональному и карьерному росту!Ведущая федеральная розничная сеть магазинов «Пятерочка» приглашает на работу:', 
    'Должностные обязанности:',
    'В крупную, динамично развивающуюся транспортную компанию требуется амбициозный, не боящийся трудностей, целеустремленный и просто позитивный человек, который любит заниматься активными продажами, поиском клиентов, преодолевать трудности и развиваться как профессионал в этой области. Требования:',
    'Наши пожелания:',
    'Мы ожидаем:',
    'ЦЕЛЬ РАБОТЫ:',
    'Чем предстоит заниматься:',
    'Предоставляем нашим сотрудникам:',
    'Ищете работу в перспективной, быстро развивающейся Компании?'
]

In [7]:
def df_preproc (df, columns = None):
    df = df.drop('id', axis=1)
    df['description'] = df['description'].apply(lambda x: re.sub('<.*?>', ' ', x))
    
    for words in deleted_words:
        df['description'] = df['description'].map(lambda x: x.replace(words, ''))

    all_text = []
    for names in df['name']:
        all_text.append(names)
    
    index = 0
    for description in df['description']:
        all_text[index]+=' '+description
        index += 1
    
    if columns is None:
        count_vect = TfidfVectorizer(tokenizer=getMeaningfullWords, ngram_range=[1,2], min_df=0.01, max_df=0.8, norm='l2', smooth_idf=True, sublinear_tf=True)
    else:
        count_vect = TfidfVectorizer(tokenizer=getMeaningfullWords, ngram_range=[1,2], vocabulary=columns, norm='l2', smooth_idf=True, sublinear_tf=True)
    
    matrix_tfidf = count_vect.fit_transform(all_text).toarray()
    
    df = pd.DataFrame(matrix_tfidf, columns=count_vect.get_feature_names())
    
    return df 

In [8]:
df_train = df_preproc(df_train)

In [ ]:
# CorrKoef = df_train.corr()
# FieldDrop = [i for i in CorrKoef if CorrKoef[i].isnull().drop_duplicates().values[0]]
# CorField = []
# for i in CorrKoef:
#     for j in CorrKoef.index[CorrKoef[i] > 0.9]:
#         if i != j and j not in CorField and i not in CorField:
#             CorField.append(j)
#             print("{}-->{}: r^2={}".format(i, j, CorrKoef[i][CorrKoef.index==j].values[0]))

In [9]:
X = df_train
y = target

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

In [11]:
# lr = Pipeline([StandardScaler(), LogisticRegression()])
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
lr_predict = lr.predict(X_test)
lr_predict_proba = lr.predict_proba(X_test) 
roc_auc_score(y_test, lr_predict_proba[:,1])

0.9894379643257674

In [13]:
rf = RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [14]:
rf_predict = rf.predict(X_test)
rf_predict_proba = rf.predict_proba(X_test) 
roc_auc_score(y_test, rf_predict_proba[:,1])

0.9923233738710031

In [ ]:
df_test = pd.read_csv('test.csv', encoding='utf-8', sep='\t')

In [ ]:
df_id = df_test['id']

In [ ]:
df_test = df_preproc(df_test, df_train.columns)

In [ ]:
model_predict = rf.predict(df_test)
model_for_file = pd.DataFrame(columns=['id', 'target'])

model_for_file['id'] = df_id
model_for_file['target'] = model_predict
model_for_file.to_csv('predict.csv', index = False)